Advent of Code 2024
===================
# Day 1
## Part 1

In [1]:
t1 = [3, 4, 2, 1, 3, 3]
t2 = [4, 3, 5, 3, 9, 3]
sum([abs(x-y) for x, y in zip(sorted(t1), sorted(t2))])

11

In [2]:
with open('1.txt') as file:
    pairs = [(int(x), int(y)) for x, y in (line.split() for line in file)]
l1, l2 = zip(*pairs)
sum([abs(x-y) for x, y in zip(sorted(l1), sorted(l2))])

2375403

## Part 2

In [49]:
[x * len([y for y in t2 if x == y]) for x in t1]

[9, 4, 0, 0, 9, 9]

sum([x * len([y for y in l2 if x == y]) for x in l1])

-----
# Day 2
see [J solution](aoc2024.ijs)
-----
# Day 3
## Part 1

In [8]:
import re
s = 'xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))'

In [22]:
sum([int(m[1]) * int(m[2]) for m in re.finditer(r'mul\((\d+),(\d+)\)', s)])

161

In [56]:
def eval_mult(s):
    return sum([int(m[1]) * int(m[2]) for m in re.finditer(r'mul\((\d+),(\d+)\)', s)])

with open('3.txt') as input:
    s = input.read()
eval_mult(s)

173419328

## Part 2

In [60]:
s = "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"

In [59]:
def do_blocks(ss):
    ds = []
    do = True
    for s in ss:
        if s == 'do()':
            do = True
        elif s == "don't()":
            do = False
        else:
            if do:
                ds.append(s)
    return ds

In [54]:
do_blocks(re.split(r"(do\(\)|don't\(\))", s))

['xmul(2,4)&mul[3,7]!^', '?mul(8,5))']

In [62]:
sum([eval_mult(t) for t in do_blocks(re.split(r"(do\(\)|don't\(\))", s))])

48

In [64]:
with open('3.txt') as input:
    s = input.read()
sum([eval_mult(t) for t in do_blocks(re.split(r"(do\(\)|don't\(\))", s))])

90669332

-----
# Day 4


In [67]:
m = [[1,2,3], [4,5,6], [7,8,9]]
m[1][2]

6

-----
# Day 5
TODO

-----
# Day 6
## Part 1

In [86]:
class WalkableMap:
    def __init__(self, map):
        self.map = map
        self.max_x0 = len(map)
        self.max_x1 = len(map[0])
        self.start = self.__find_guard()
        self.__reset()
        self.__set('X')
        self.count = 1

        
    def __find_guard(self):
        '''
        Detect the starting position '^' in the given map
        '''
        return next((i,s.index('^')) for i,s in enumerate(self.map) if '^' in s)

    def __reset(self):
        self.x = self.start
        self.v = (-1, 0)
        self.path = [(self.x, self.v)]
        self.stuck = False

    def __set(self, c, x=None):
        if x == None:
            x = self.x
        x0, x1 = x
        l = list(self.map[x0])
        l[x1] = c
        self.map[x0] = ''.join(l)

    def step(self):
        if self.done():
            return
        x0, x1 = self.x
        y = x0 + self.v[0], x1 + self.v[1]
        if self.__exited(y):
            self.x = y
        elif self.__peek(y) == '#':
            self.v = self.v[1], -self.v[0]
        else:
            if (y, self.v) in self.path:
                self.stuck = True
            else:
                self.path.append((y, self.v))
            self.x = y
            if self.__peek() == '.':
                self.__set('X')
                self.count += 1

    def done(self):
        return self.exited() or self.stuck
        
    def exited(self):
        return self.__exited(self.x)

    def __exited(self, p):
        return p[0] < 0 or p[0] >= self.max_x0 or p[1] < 0 or p[1] >= self.max_x1

    def __peek(self, y=None):
        if y == None:
            y = self.x
        return self.map[y[0]][y[1]]
    
    def route(self):
        '''
        Keep steppping until the guard exits the map

        Returns the number of locations visited.
        '''
        while not self.done():
            self.step()
        return self.count

    def traps(self):
        '''
        Count the number of ways to trap the guard
        '''
        cnt = 0
        for x0 in range(self.max_x0):
            for x1 in range(self.max_x1):
                if (x0,x1) != self.start and self.__peek((x0, x1)) != '#':
                    self.__set('#', (x0,x1))
                    self.__reset()
                    self.route()
                    if self.stuck:
                        cnt += 1
                    self.__set('.', (x0,x1))
        return cnt
        
        
wm = WalkableMap('''
....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
'''.split())

wm.route()

41

In [87]:
with open('6.txt') as input:
    txt6 = input.read()
WalkableMap(txt6.split()).route()

5153

## Part 2

In [88]:
wm.traps()

6

In [89]:
WalkableMap(txt6.split()).traps()

1711

# Day 7
## Part 1

In [124]:
ss = '''190: 10 19
3267: 81 40 27
83: 17 5
156: 15 6
7290: 6 8 6 15
161011: 16 10 13
192: 17 8 14
21037: 9 7 18 13
292: 11 6 16 20'''.split('\n')

def parse7(s):
    r, ns = s.split(': ')
    return int(r), [int(x) for x in ns.split()]

def chk7(eq, concat=False):
    r, xs = eq
    ys = xs[1:]
    for fs in ops_lists(len(ys), concat):
        x = xs[0]
        for f, y in zip(fs, ys):
            x = f(x, y)
        if x == r:
            return True
    return False

def ops_lists(n, concat):
    if n == 0:
        return [[]]
    else:
        ls = ops_lists(n-1, concat)
        r = [l + [lambda x,y: x+y] for l in ls] + [l + [lambda x,y: x*y] for l in ls]
        if concat:
            r += [l + [lambda x, y: int(str(x) + str(y))] for l in ls]
        return r

In [125]:
chk7((3267, [81, 40, 27]))

True

In [126]:
sum([x for x,xs in list(map(parse7, ss)) if chk7((x,xs))])

3749

In [127]:
with open('7.txt') as input:
    txt7 = [s for s in input.read().split('\n') if len(s) > 0]
sum([x for x,xs in list(map(parse7, txt7)) if chk7((x,xs))])

5030892084481

## Part 2

In [130]:
sum([x for x,xs in list(map(parse7, ss)) if chk7((x,xs), True)])

11387

In [131]:
with open('7.txt') as input:
    txt7 = [s for s in input.read().split('\n') if len(s) > 0]
sum([x for x,xs in list(map(parse7, txt7)) if chk7((x,xs), True)])

91377448644679